<a href="https://colab.research.google.com/github/boiBASH/Tolaram_Project/blob/main/tolaram_recomender_unmasked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Load your dataset (replace with the actual path in your Drive)
file_path = "/content/drive/MyDrive/TOLARAM/Data sample analysis.csv"
df = pd.read_csv(file_path, encoding='latin-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-7cdf484f43a5>:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


In [2]:
# Step 1: Convert Delivered_date to datetime
df['Delivered_date'] = pd.to_datetime(df['Delivered_date'], errors='coerce', dayfirst=True)
df

,Brand,Branch,Designation,SKU_Code,Salesman_Code,Salesman_Name,Customer_Name,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value
0,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807259,ANOSIKE MIRACLE CHIMECHEFULAM,Eze store,8060733751,11432770,2024-08-05,3.0,"46,500"
1,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11565436,2024-08-26,1.0,"15,500"
2,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11727113,2024-09-17,1.0,"15,500"
3,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Cynthia chi,8060359775,11756045,2024-09-20,1.0,"15,500"
4,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Daddy Josephine,9075013565,11585122,2024-08-29,1.0,"15,500"
...,...,...,...,...,...,...,...,...,...,...,...,...
108820,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,House 1 super store,8101599308,12090661,2024-11-01,1.0,"56,800"
108821,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Loveth ejiro,8139407516,12282897,2024-11-22,1.0,"33,000"
108822,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Mrs Jennifer,8060468927,12067581,2024-10-30,4.0,"201,600"
108823,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,TWINNY SQUARE ENTERPRISES,8055148535,12032444,2024-10-25,1.0,"50,400"


In [3]:

# Step 2: Clean and convert Redistribution Value (remove commas and convert to float)
df['Redistribution Value'] = df['Redistribution Value'].str.replace(',', '', regex=False).astype(float)

# Step 3: Fill missing 'Delivered Qty' with 0 (or could be dropped if preferred)
df['Delivered Qty'] = df['Delivered Qty'].fillna(0)


# Confirm changes
df.dtypes


,0
Brand,object
Branch,object
Designation,object
SKU_Code,object
Salesman_Code,object
Salesman_Name,object
Customer_Name,object
Customer_Phone,int64
Order_Id,int64
Delivered_date,datetime64[ns]


In [4]:
# 1. Total amount spent per purchase
df['Total_Amount_Spent'] = df['Redistribution Value'] * df['Delivered Qty']

# 2. Average product cost per SKU
sku_grouped = df.groupby('SKU_Code').agg({
    'Redistribution Value': 'sum',
    'Delivered Qty': 'sum'
}).rename(columns={
    'Redistribution Value': 'Total_Value',
    'Delivered Qty': 'Total_Quantity'
})
sku_grouped['Average_Product_Cost'] = sku_grouped['Total_Value'] / sku_grouped['Total_Quantity']

# Merge back to main DataFrame
df = df.merge(sku_grouped[['Average_Product_Cost']], on='SKU_Code', how='left')
df

,Brand,Branch,Designation,SKU_Code,Salesman_Code,Salesman_Name,Customer_Name,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value,Total_Amount_Spent,Average_Product_Cost
0,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807259,ANOSIKE MIRACLE CHIMECHEFULAM,Eze store,8060733751,11432770,2024-08-05,3.0,46500.0,139500.0,15500.000000
1,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11565436,2024-08-26,1.0,15500.0,15500.0,15500.000000
2,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11727113,2024-09-17,1.0,15500.0,15500.0,15500.000000
3,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Cynthia chi,8060359775,11756045,2024-09-20,1.0,15500.0,15500.0,15500.000000
4,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Daddy Josephine,9075013565,11585122,2024-08-29,1.0,15500.0,15500.0,15500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108820,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,House 1 super store,8101599308,12090661,2024-11-01,1.0,56800.0,56800.0,49057.746991
108821,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Loveth ejiro,8139407516,12282897,2024-11-22,1.0,33000.0,33000.0,49057.746991
108822,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Mrs Jennifer,8060468927,12067581,2024-10-30,4.0,201600.0,806400.0,49057.746991
108823,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,TWINNY SQUARE ENTERPRISES,8055148535,12032444,2024-10-25,1.0,50400.0,50400.0,49057.746991


In [5]:
import pandas as pd
from pprint import pprint # Keep pprint for displaying the structured output

# Make sure analyze_customer_purchases_extended is defined first

def analyze_customer_purchases_extended(df, customer_phone):
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        return f"No data found for customer phone: {customer_phone}"

    # Ensure date is sorted
    customer_df.sort_values('Delivered_date', inplace=True)

    # Add Month column
    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    # Basic customer information
    customer_name = customer_df['Customer_Name'].iloc[0] if not customer_df.empty else 'N/A'

    # 1. Total Brands Bought
    brands_bought = customer_df['Brand'].unique().tolist()
    total_brands_bought = len(brands_bought)

    # 2. SKUs of each Brand Bought
    brand_skus = customer_df.groupby('Brand')['SKU_Code'].unique().apply(list).to_dict()

    # 3. Purchase Summary by Brand
    purchase_summary_by_brand = {}
    for brand in brands_bought:
        brand_df = customer_df[customer_df['Brand'] == brand]
        last_purchase_date = brand_df['Delivered_date'].max().strftime('%Y-%m-%d') if not brand_df.empty else 'N/A'
        total_quantity = brand_df['Delivered Qty'].sum()
        total_spent = brand_df['Total_Amount_Spent'].sum()
        purchase_summary_by_brand[brand] = {
            'Last Purchase Date': last_purchase_date,
            'Total Quantity Bought': total_quantity,
            'Total Amount Spent': round(total_spent, 2)
        }

    # 4. Purchase Summary for Brand SKU
    purchase_summary_by_brand_sku = {}
    for brand, skus in brand_skus.items():
        purchase_summary_by_brand_sku[brand] = {}
        for sku in skus:
            sku_df = customer_df[(customer_df['Brand'] == brand) & (customer_df['SKU_Code'] == sku)]
            if not sku_df.empty:
                last_purchase = sku_df['Delivered_date'].max().strftime('%Y-%m-%d')
                # Handle potential division by zero if a Month group has no quantity/spend
                monthly_qty_series = sku_df.groupby('Month')['Delivered Qty'].sum()
                avg_monthly_qty = monthly_qty_series.mean().round(2) if not monthly_qty_series.empty else 0.0

                monthly_spend_series = sku_df.groupby('Month')['Total_Amount_Spent'].sum()
                avg_monthly_spend = monthly_spend_series.mean().round(2) if not monthly_spend_series.empty else 0.0

                purchase_summary_by_brand_sku[brand][sku] = {
                    'Last Purchase Date': last_purchase,
                    'Avg Monthly Quantity': avg_monthly_qty,
                    'Avg Monthly Spend': avg_monthly_spend
                }
            else:
                purchase_summary_by_brand_sku[brand][sku] = {
                    'Last Purchase Date': 'N/A',
                    'Avg Monthly Quantity': 0.0,
                    'Avg Monthly Spend': 0.0
                }


    # 5. Salesman Analysis - Corrected Calculation
    most_sold_salesman_info = 'N/A' # Initialize as N/A
    salesman_designation = 'N/A'

    if 'Salesman_Name' in customer_df.columns and 'Order_Id' in customer_df.columns and not customer_df.empty:
        # Group by Salesman_Name and count unique Order_Id values
        salesman_unique_order_counts = customer_df.groupby('Salesman_Name')['Order_Id'].nunique()

        if not salesman_unique_order_counts.empty and salesman_unique_order_counts.max() > 0:
            # Find the salesman with the maximum unique order count
            most_sold_salesman_name = salesman_unique_order_counts.idxmax()
            most_sold_salesman_count = salesman_unique_order_counts.max()

            # Format the salesman info with count in brackets
            most_sold_salesman_info = f"{most_sold_salesman_name} ({int(most_sold_salesman_count)} orders)"

            # Assuming 'Designation' is the correct column name for salesman designation
            # Ensure 'Designation' column exists before accessing it
            if 'Designation' in customer_df.columns and not customer_df[customer_df['Salesman_Name'] == most_sold_salesman_name].empty:
                 salesman_designation = customer_df[customer_df['Salesman_Name'] == most_sold_salesman_name]['Designation'].iloc[0]
            else:
                 salesman_designation = 'Designation data not available'


    # Format the output for better readability
    report = {
        'Customer Analysis Report': {
            'Customer Phone': customer_phone,
            'Customer Name': customer_name,
            'Overview': {
                'Total Unique Brands Bought': total_brands_bought,
                'Brands Bought List': brands_bought,
                # Updated key and value to include count in the name
                'Top Salesperson': most_sold_salesman_info,
                'Salesperson Designation': salesman_designation
            },
            'Brand Level Summary': purchase_summary_by_brand,
            'Brand SKU Level Summary': purchase_summary_by_brand_sku,
            'SKUs Grouped by Brand': brand_skus
        }
    }

    return report


from IPython.display import display # Import display for better table rendering in notebooks
def analyze_customer_purchases_extended_table(df, customer_phone):
    # First, get the customer_df to check for existence and get basic info efficiently
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        print(f"No data found for customer phone: {customer_phone}")
        return

    # Ensure date is sorted
    customer_df.sort_values('Delivered_date', inplace=True)

    # Add Month column
    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    analysis_report = analyze_customer_purchases_extended(df, customer_phone)

    # Safely access data from the structured report dictionary
    report_data = analysis_report.get('Customer Analysis Report', {})
    overview_data_dict = report_data.get('Overview', {})
    brand_summary_data_dict = report_data.get('Brand Level Summary', {})
    brand_sku_summary_data_dict = report_data.get('Brand SKU Level Summary', {})
    skus_grouped_data_dict = report_data.get('SKUs Grouped by Brand', {})
    customer_name = report_data.get('Customer Name', 'N/A')
    brands_bought = overview_data_dict.get('Brands Bought List', [])
    top_salesperson_info = overview_data_dict.get('Top Salesperson', 'N/A')
    salesperson_designation = overview_data_dict.get('Salesperson Designation', 'N/A')
    total_brands_bought = overview_data_dict.get('Total Unique Brands Bought', 0)

    # --- Additional Overview Calculations ---

    # Calculate Total Order Count
    # Count the number of unique Order_Id for this customer
    total_order_count = 0
    if 'Order_Id' in customer_df.columns:
        total_order_count = customer_df['Order_Id'].nunique()
    else:
        # Fallback to row count if Order_Id is not available, though unique Order_Id is better
        total_order_count = len(customer_df)
        print("Warning: 'Order_Id' column not found. Using total rows as approximate order count.")


    # Customer Branch
    customer_branch = 'N/A'
    if 'Branch' in customer_df.columns and not customer_df.empty:
        unique_branches = customer_df['Branch'].unique()
        if len(unique_branches) == 1:
            customer_branch = unique_branches[0]
        elif len(unique_branches) > 1:
            customer_branch = ", ".join(unique_branches)
        else:
             customer_branch = 'N/A (Branch data missing)'
    else:
        customer_branch = 'N/A (Branch column missing)'


    # --- Create Overview Table ---
    overview_data = {
        'Metric': [
            'Customer Phone',
            'Customer Name',
            'Customer Branch', # Added Customer Branch
            'Total Unique Brands Bought',
            'Total Order Count', # Added Total Order Count
            'Top Salesperson',
            'Salesperson Designation'
        ],
        'Value': [
            customer_phone,
            customer_name,
            customer_branch, # Added Customer Branch value
            total_brands_bought,
            total_order_count, # Added Total Order Count value
            top_salesperson_info,
            salesperson_designation
        ]
    }
    overview_df = pd.DataFrame(overview_data)

    print("--- Customer Overview ---")
    display(overview_df) # Use display for better formatting in notebooks

    print("\n--- Brands Bought ---")
    # Create a DataFrame for the list of brands bought
    brands_df = pd.DataFrame({'Brands Bought': brands_bought})
    display(brands_df)


    # --- Create Brand Level Summary Table ---
    print("\n--- Brand Level Summary ---")
    purchase_summary_by_brand_list = []
    for brand, summary in brand_summary_data_dict.items():
         purchase_summary_by_brand_list.append({
             'Brand': brand,
             'Last Purchase Date': summary.get('Last Purchase Date', 'N/A'),
             'Total Quantity Bought': summary.get('Total Quantity Bought', 0),
             'Total Amount Spent': summary.get('Total Amount Spent', 0.0)
         })
    brand_summary_df = pd.DataFrame(purchase_summary_by_brand_list)
    # Ensure Brand is the first column
    if 'Brand' in brand_summary_df.columns:
        cols = ['Brand'] + [col for col in brand_summary_df.columns if col != 'Brand']
        brand_summary_df = brand_summary_df[cols]
    display(brand_summary_df)


    # --- Create Brand SKU Level Summary Table(s) ---
    print("\n--- Brand SKU Level Summary ---")
    for brand, skus_summary in brand_sku_summary_data_dict.items():
        sku_summary_list = []
        for sku, summary in skus_summary.items():
            sku_summary_list.append({
                'SKU Code': sku,
                'Last Purchase Date': summary.get('Last Purchase Date', 'N/A'),
                'Avg Monthly Quantity': summary.get('Avg Monthly Quantity', 0.0),
                'Avg Monthly Spend': summary.get('Avg Monthly Spend', 0.0)
            })
        sku_summary_df = pd.DataFrame(sku_summary_list)
        if not sku_summary_df.empty:
             print(f"\nBrand: {brand}")
             # Ensure 'SKU Code' is the first column
             if 'SKU Code' in sku_summary_df.columns:
                 cols = ['SKU Code'] + [col for col in sku_summary_df.columns if col != 'SKU Code']
                 sku_summary_df = sku_summary_df[cols]
             display(sku_summary_df)


    # --- Create SKUs Grouped by Brand Table(s) ---
    print("\n--- SKUs Grouped by Brand ---")
    for brand, skus in skus_grouped_data_dict.items():
        if skus: # Check if the list of SKUs is not empty
             print(f"\nBrand: {brand}")
             skus_df = pd.DataFrame({'SKU Codes': skus})
             display(skus_df)

In [9]:
sample_customer = df['Customer_Phone'].iloc[99]
analyze_customer_purchases_extended_table(df, sample_customer)

--- Customer Overview ---


,Metric,Value
0,Customer Phone,8032343938
1,Customer Name,Nkem Stores
2,Customer Branch,LAGOS 1
3,Total Unique Brands Bought,7
4,Total Order Count,58
5,Top Salesperson,NKECHI ROBERT (34 orders)
6,Salesperson Designation,FIELD EXECUTIVE



--- Brands Bought ---


,Brands Bought
0,ADDMIE
1,INDOMIE
2,KELLOGGS
3,MUNCH IT
4,POWER OIL
5,MINIMIE
6,COLGATE



--- Brand Level Summary ---


...

,SKU Codes
0,10000423



Brand: MUNCH IT


,SKU Codes
0,10002849
1,10002866



Brand: POWER OIL


,SKU Codes
0,10000539
1,10000533
2,VOB4500_
3,10000536


In [7]:
sample_customer = df['Customer_Phone'].iloc[412]
analyze_customer_purchases_extended_table(df, sample_customer)

--- Customer Overview ---


,Metric,Value
0,Customer Phone,8103939788
1,Customer Name,Juliet
2,Customer Branch,LAGOS 1
3,Total Unique Brands Bought,5
4,Total Order Count,34
5,Top Salesperson,BUNMI FADUNSIN (10 orders)
6,Salesperson Designation,FIELD EXECUTIVE



--- Brands Bought ---


,Brands Bought
0,KELLOGGS
1,MUNCH IT
2,INDOMIE
3,ADDMIE
4,MINIMIE



--- Brand Level Summary ---


...

,SKU Codes
0,10002826
1,10002771
2,10002843
3,10002773
4,10002823
5,10002832
6,10002835
7,10002830
8,10002838
9,10002827



Brand: MINIMIE


,SKU Codes
0,10000423
1,10003401



Brand: MUNCH IT


,SKU Codes
0,10002849
1,10002857
2,10002860
3,10002866
4,10002836


In [8]:
sample_customer = df['Customer_Phone'].iloc[9999]
analyze_customer_purchases_extended_table(df, sample_customer)

--- Customer Overview ---


,Metric,Value
0,Customer Phone,8136149255
1,Customer Name,Mr pelumi
2,Customer Branch,LAGOS 1
3,Total Unique Brands Bought,3
4,Total Order Count,12
5,Top Salesperson,BASIRAT BOLANLE ABDULAZEEZ (6 orders)
6,Salesperson Designation,FIELD EXECUTIVE



--- Brands Bought ---


,Brands Bought
0,KELLOGGS
1,COLGATE
2,POWER OIL



--- Brand Level Summary ---


...

,SKU Codes
0,72TPHRB130_
1,TPDCP130_
2,48TCGC120_
3,10002959
4,10002804
5,10002972



Brand: KELLOGGS


,SKU Codes
0,10002830
1,10002832
2,10002835
3,10002838
4,10002843
5,10003395



Brand: POWER OIL


,SKU Codes
0,10000539
